In [51]:
import pandas as pd
import numpy as np
import aqi

In [12]:
df = pd.read_csv('rawdata/yearly_data1.csv')

df.head(5)

,ValidDate,ValidTime,AQSID,SiteName,GMTOffset,MeasurementType,MeasurementUnit,RawMeasurement,DataSource
0,01/01/14,00:00,250250002,BOSTON-KENMORE,-5,NO2,PPB,26.0,Massachusetts Dept. of Environmental Protection
1,01/01/14,00:00,250250002,BOSTON-KENMORE,-5,CO,PPM,0.2,Massachusetts Dept. of Environmental Protection
2,01/01/14,00:00,250250002,BOSTON-KENMORE,-5,SO2,PPB,3.0,Massachusetts Dept. of Environmental Protection
3,01/01/14,00:00,250250002,BOSTON-KENMORE,-5,NOX,PPB,39.0,Massachusetts Dept. of Environmental Protection
4,01/01/14,00:00,250250002,BOSTON-KENMORE,-5,NO,PPB,13.0,Massachusetts Dept. of Environmental Protection


In [13]:
df = df.drop(['GMTOffset', 'ValidTime', 'DataSource'], axis=1)

df.head(5)

,ValidDate,AQSID,SiteName,MeasurementType,MeasurementUnit,RawMeasurement
0,01/01/14,250250002,BOSTON-KENMORE,NO2,PPB,26.0
1,01/01/14,250250002,BOSTON-KENMORE,CO,PPM,0.2
2,01/01/14,250250002,BOSTON-KENMORE,SO2,PPB,3.0
3,01/01/14,250250002,BOSTON-KENMORE,NOX,PPB,39.0
4,01/01/14,250250002,BOSTON-KENMORE,NO,PPB,13.0


In [14]:
df = df[(df['MeasurementType'] == 'NO2') | (df['MeasurementType'] == 'PM2.5') | (df['MeasurementType'] == 'OZONE') ]

df.head(5)

,ValidDate,AQSID,SiteName,MeasurementType,MeasurementUnit,RawMeasurement
0,01/01/14,250250002,BOSTON-KENMORE,NO2,PPB,26.0
9,01/01/14,250250041,Boston - Long Island,OZONE,PPB,21.0
10,01/01/14,250250041,Boston - Long Island,NO2,PPB,13.0
15,01/01/14,250250042,Boston - Roxbury,OZONE,PPB,8.0
16,01/01/14,250250042,Boston - Roxbury,NO2,PPB,29.0


In [53]:
df[df["MeasurementType"] == "PM2.5"].value_counts()

ValidDate  AQSID      SiteName            MeasurementType  MeasurementUnit  RawMeasurement
12/01/19   250250044  Boston-Von Hillern  PM2.5            UG/M3            2.0               1
01/01/14   250250042  Boston - Roxbury    PM2.5            UG/M3            18.1              1
           250250043  Boston - North End  PM2.5            UG/M3            29.5              1
           250250044  Boston-Von Hillern  PM2.5            UG/M3            18.8              1
01/01/15   250250042  Boston - Roxbury    PM2.5            UG/M3            11.6              1
                                                                                             ..
01/01/17   250250044  Boston-Von Hillern  PM2.5            UG/M3            5.0               1
01/01/18   250250042  Boston - Roxbury    PM2.5            UG/M3            9.0               1
           250250043  Boston - North End  PM2.5            UG/M3            19.0              1
           250250044  Boston-Von Hillern  PM2

In [54]:
groups = df.groupby(['ValidDate', 'SiteName', 'AQSID']).indices
new_dict = {
    "ValidDate": [],
    "AQSID": [],
    "SiteName": [],
    "OZONE_AQI": [],
    "PM25_AQI": [],
    "NO2_AQI": [],
}

for (date, site, id), indices in groups.items():
    new_dict["ValidDate"].append(date)
    new_dict["SiteName"].append(site)
    new_dict["AQSID"].append(id)

    ozone_aqi = np.nan
    pm25_aqi = np.nan
    no2_aqi = np.nan

    for i in indices:
        row = df.iloc[i]

        if row['MeasurementType'] == "NO2":
            if row['MeasurementUnit'] == "PPB":
                no2_aqi = aqi.to_iaqi(aqi.POLLUTANT_NO2_1H, row['RawMeasurement'], algo=aqi.ALGO_EPA)
            else:
                ppb = row['RawMeasurement'] * 0.51 # Convert from ug/m3 to ppb
                no2_aqi = aqi.to_iaqi(aqi.POLLUTANT_NO2_1H, ppb, algo=aqi.ALGO_EPA)

        elif row['MeasurementType'] == "OZONE":
            if row['MeasurementUnit'] == "PPB":
                ppm = row['RawMeasurement'] * 0.001 # Convert from ppb to ppm
                ozone_aqi = aqi.to_iaqi(aqi.POLLUTANT_O3_8H, ppm, algo=aqi.ALGO_EPA)
            else:
                ppm = row['RawMeasurement'] * 0.001 * 0.49 # Convert from ug/m3 to ppm
                ozone_aqi = aqi.to_iaqi(aqi.POLLUTANT_O3_8H, ppm, algo=aqi.ALGO_EPA)

        elif row['MeasurementType'] == "PM2.5":
            if row['MeasurementUnit'] == "UG/M3":
                no2_aqi = aqi.to_iaqi(aqi.POLLUTANT_PM25, row['RawMeasurement'], algo=aqi.ALGO_EPA)
    
    new_dict["OZONE_AQI"].append(ozone_aqi)
    new_dict["PM25_AQI"].append(pm25_aqi)
    new_dict["NO2_AQI"].append(no2_aqi)


new_df = pd.DataFrame(new_dict)
new_df.head(10)

IndexError: list index out of range